In [1]:
# RSI PowerZones Weekly
import gta_indicator
import gta_plot
import gta_prices
import gta_signals
import gta_statistics
import gta_trades
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# Trade parameters.
exchange = 'LSE'
tidm = 'IBPO'
timeframe = 'Weekly'
filename = f'{exchange}_{tidm}_prices.csv'
history_start = '2010-09-14'
p1 = 48 # System 1 look back period.
p2 = 24 # System 2 look back period.
p3 = 12 # System 3 look back period.
p4 = 6  # System 4 look back period.
look_back = (p1, p2, p3, p4)
strategy = f'Donchian'
position_size = 7500  # Position size in major currency unit.
charges = 11.95  # Commission per trade.
stamp_duty = 0.0  # Stamp Duty percentage.
stop_loss = 20  # Stop Loss percentage.

In [3]:
# Import weekly closing prices.
df = gta_prices.weekly(exchange, tidm, history_start)

In [4]:
# Calculate Donchian channel for each System.
for period in look_back:
    sys = look_back.index(period) + 1
    dc = gta_indicator.donchian(df, sys, period)
    df = df.join(dc)

In [5]:
# System 1 long trades.
df['buy'] = np.where(df.close > df.upr1, 1, 0) # Check for close above upper.
df['sell1'] = np.where(df.close < df.mid1, 1, 0) # Check for close below mid.
df['ss1'] = gta_signals.state_signal(df.buy, df.sell1, p1) # Trade state signals.
df['en1'] = np.where(np.logical_and(df.ss1 == 1, df.ss1.shift(periods=1) == 0), 1, 0)
df['ex1'] = np.where(np.logical_and(df.ss1 == 0, df.ss1.shift(periods=1) == 1), 1, 0)

# System 2 long trades.
df['sell2'] = np.where(df.close < df.mid2, 1, 0)
df['ss2'] = gta_signals.state_signal(df.buy, df.sell2, p2)
df['ex2'] = np.where(np.logical_and(df.ss2 == 0, df.ss2.shift(periods=1) == 1), 1, 0)

# System 3 long trades.
df['sell3'] = np.where(df.close < df.mid3, 1, 0)
df['ss3'] = gta_signals.state_signal(df.buy, df.sell3, p3)
df['ex3'] = np.where(np.logical_and(df.ss3 == 0, df.ss3.shift(periods=1) == 1), 1, 0)

# System 4 long trades.
df['sell4'] = np.where(df.close < df.mid4, 1, 0)
df['ss4'] = gta_signals.state_signal(df.buy, df.sell4, p4)
df['ex4'] = np.where(np.logical_and(df.ss4 == 0, df.ss4.shift(periods=1) == 1), 1, 0)

df

,open,high,low,close,upr1,lwr1,mid1,upr2,lwr2,mid2,...,ex1,sell2,ss2,ex2,sell3,ss3,ex3,sell4,ss4,ex4
date,,,,,,,,,,,,,,,,,,,,,
2010-09-17,1.225,1.400,1.200,1.375,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2010-09-24,1.375,1.375,1.350,1.375,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2010-10-01,1.375,1.400,1.350,1.400,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2010-10-08,1.395,1.540,1.395,1.540,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2010-10-15,1.540,1.550,1.540,1.540,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-25,3.910,4.000,3.780,3.860,3.94,2.48,3.21,3.94,2.48,3.21,...,0,0,0,0,0,0,0,0,0,0
2022-04-01,3.980,4.000,3.840,3.860,4.00,2.48,3.24,4.00,2.54,3.27,...,0,0,0,0,0,0,0,0,0,0
2022-04-08,3.950,3.990,3.820,3.920,4.00,2.48,3.24,4.00,2.54,3.27,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# Trade list.
# td1 = gta_trades.trade_list(df.close, df.en1,  df.ex1)
# td2 = gta_trades.trade_list(df.close, df.en1,  df.ex2)
# td3 = gta_trades.trade_list(df.close, df.en1,  df.ex3)
# td4 = gta_trades.trade_list(df.close, df.en1,  df.ex4)
# td = pd.concat([td1, td2, td3, td4], axis=0)
# td4 = td4.sort_index()
# td4